# Install Required Libraries

In [ ]:
%pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain  

In [ ]:
import os
os.environ.get['LANGCHAIN_TRACING_V2']
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = <your-api-key>